<a href="https://colab.research.google.com/github/ScriptSherpa/TensorFlow-Basic-to-Advanced---100-Projects-in-100-Days/blob/main/SFH_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
umairshafique_sfh_dataset_path = kagglehub.dataset_download('umairshafique/sfh-dataset')

print('Data source import complete.')


## import the libraries and split the data into train,test,and validation

In [ ]:
!pip install split-folders

In [ ]:
import tensorflow as tf
import numpy as np
import os
import splitfolders

input_folder = '/kaggle/input/sfh-dataset/3 Classification'

output_folder = 'train_test_val'

splitfolders.ratio(input_folder, output=output_folder, ratio=(0.7, 0.15, 0.15))

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

Copying files: 7393 files [00:11, 667.14 files/s]


## build the model layers (CNN)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


## data augmentation

In [ ]:
# Define paths for training, validation, and test directories
train_dir = os.path.join(output_folder, 'train')
val_dir = os.path.join(output_folder, 'val')
test_dir = os.path.join(output_folder, 'test')

# Data augmentation for the training data
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
).flow_from_directory(train_dir,
                      color_mode='rgb',
                      target_size=(100, 100),
                      batch_size=128,
                      class_mode='categorical')

# Validation data generator
val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
).flow_from_directory(val_dir,
                      color_mode='rgb',
                      target_size=(100, 100),
                      batch_size=16,
                      class_mode='categorical')

# Test data generator
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255
).flow_from_directory(test_dir,
                      color_mode='rgb',
                      target_size=(100, 100),
                      batch_size=16,
                      class_mode='categorical')


Found 5174 images belonging to 2 classes.
Found 1108 images belonging to 2 classes.
Found 1111 images belonging to 2 classes.


## fit the model and save the best results

In [ ]:
# Fit the model and save the best results
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.keras',
                                                monitor='val_loss',
                                                verbose=1,
                                                save_best_only=True,
                                                mode='min')

history = model.fit(train_generator,
                    epochs=25,
                    validation_data=val_generator,
                    callbacks=[checkpoint])


Epoch 1/25


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 2/41 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.6953 - loss: 0.6481  

I0000 00:00:1747023755.345560     112 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7559 - loss: 0.6069
Epoch 1: val_loss improved from inf to 0.55101, saving model to model.keras
41/41 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.7560 - loss: 0.6062 - val_accuracy: 0.7617 - val_loss: 0.5510
Epoch 2/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7716 - loss: 0.5406
Epoch 2: val_loss improved from 0.55101 to 0.54368, saving model to model.keras
41/41 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.7713 - loss: 0.5408 - val_accuracy: 0.7617 - val_loss: 0.5437
Epoch 3/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7651 - loss: 0.5456
Epoch 3: val_loss did not improve from 0.54368
41/41 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.7650 - loss: 0.5458 - val_accuracy: 0.7617 - val_loss: 0.5493
Epoch 4/25
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7674 - loss: 0.5381
Epoch 4: val_loss improved from 0.54368 to 0.52836, saving model to model.keras
41/41 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 

## get the accuracy for the test data


In [ ]:
from tensorflow.keras.models import load_model

# Load the best saved model
model = load_model('model.keras')
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


2025-05-12 06:22:42.197101: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-12 06:22:42.197251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-12 06:22:42.326457: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ValueError: File not found: filepath=model.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Get predictions and true labels
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classification report
print(classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys()))


NameError: name 'model' is not defined